<a href="https://colab.research.google.com/github/heodawoon/XAI606_GCN/blob/main/%5BXAI606%5DProtein_Enzyme_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)

1.10.0+cu111
11.1


In [2]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html


In [3]:
import random

import os.path as osp
from math import ceil

import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch_geometric
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, DenseGraphConv, dense_diff_pool, TopKPooling, dense_mincut_pool #, GATConv, GraphUNet
from torch_geometric.utils import to_dense_batch, to_dense_adj

In [4]:
# torch.manual_seed(920)

In [5]:
path = "/content"
dataset = TUDataset(path, name='PROTEINS').shuffle()
n = len(dataset)
average_nodes = int(dataset.data.x.size(0) / len(dataset))

In [6]:
# random.seed(10) # Iteration 1
# random.seed(20) # Iteration 
# random.seed(30) # Iteration 3
# random.seed(40) # Iteration 4
random.seed(50) # Iteration 5
random_index = random.sample(range(len(dataset)), len(dataset))
print(random_index)


[1018, 545, 745, 496, 969, 675, 174, 1102, 649, 459, 1106, 313, 711, 201, 1100, 654, 455, 387, 140, 673, 878, 182, 131, 684, 881, 11, 465, 691, 1111, 889, 216, 411, 906, 555, 942, 259, 552, 428, 251, 843, 240, 1016, 874, 1059, 256, 1022, 1057, 615, 190, 753, 1058, 307, 13, 733, 18, 841, 968, 1103, 332, 200, 280, 524, 770, 503, 688, 68, 96, 161, 485, 536, 862, 952, 856, 800, 962, 1091, 825, 248, 742, 105, 39, 138, 908, 1062, 629, 648, 710, 1019, 181, 617, 163, 440, 1099, 948, 1012, 114, 674, 90, 1087, 1067, 135, 51, 1005, 146, 30, 318, 702, 755, 864, 981, 364, 16, 183, 512, 54, 246, 593, 920, 724, 315, 714, 603, 27, 152, 505, 413, 442, 356, 1076, 156, 228, 875, 990, 535, 619, 289, 846, 665, 343, 927, 1095, 333, 45, 148, 1084, 854, 835, 491, 291, 129, 826, 116, 322, 870, 388, 728, 830, 261, 526, 558, 125, 101, 573, 69, 28, 109, 1061, 641, 1109, 1083, 476, 1068, 301, 803, 680, 626, 254, 801, 270, 851, 229, 587, 1071, 203, 255, 305, 253, 50, 575, 510, 189, 247, 117, 712, 283, 741, 389, 783

In [7]:
n = len(dataset) // 10 
test_dataset = dataset[random_index[:n]]
val_dataset = dataset[random_index[n:2 * n]]
train_dataset = dataset[random_index[2 * n:]]
test_loader = DataLoader(test_dataset, batch_size=8)
val_loader = DataLoader(val_dataset, batch_size=8)
train_loader = DataLoader(train_dataset, batch_size=8)

In [11]:
# # The base line code

# class Net(torch.nn.Module):
#     def __init__(self, in_channels, out_channels, hidden_channels=32):
#         super(Net, self).__init__()

#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         num_nodes = ceil(0.5 * average_nodes)
#         self.pool1 = Linear(hidden_channels, num_nodes)

#         self.conv2 = DenseGraphConv(hidden_channels, hidden_channels)
#         num_nodes = ceil(0.5 * num_nodes)
#         self.pool2 = Linear(hidden_channels, num_nodes)

#         self.conv3 = DenseGraphConv(hidden_channels, hidden_channels)

#         self.lin1 = Linear(hidden_channels, hidden_channels)
#         self.lin2 = Linear(hidden_channels, out_channels)

#     def forward(self, x, edge_index, batch):
#         # print('first x: ', x.shape)
#         x = F.relu(self.conv1(x, edge_index))
#         # print('x shape: ', x.shape) # x shape:  torch.Size([176, 32])

#         x, mask = to_dense_batch(x, batch)
#         # print('x shape: ', x.shape) # x shape:  torch.Size([8, 47, 32])
#         adj = to_dense_adj(edge_index, batch) 

#         s = self.pool1(x)
#         # print('s shape: ', s.shape) # s shape:  torch.Size([8, 47, 20])
#         x, adj, mc1, o1 = dense_diff_pool(x, adj, s, mask)
#         # print('x shape: ', x.shape) # x shape:  torch.Size([8, 20, 32])

#         x = F.relu(self.conv2(x, adj))
#         # print('x shape: ', x.shape) # x shape:  torch.Size([8, 20, 32])
#         s = self.pool2(x)
#         # print('s shape: ', s.shape) # s shape:  torch.Size([8, 20, 10])

#         x, adj, mc2, o2 = dense_diff_pool(x, adj, s)
#         # print('x shape: ', x.shape) # x shape:  torch.Size([8, 10, 32])

#         x = self.conv3(x, adj)
#         # print('x shape: ', x.shape) # x shape:  torch.Size([8, 10, 32])

#         x = x.mean(dim=1)
#         # print('x shape: ', x.shape) # x shape:  torch.Size([8, 32])
#         x = F.relu(self.lin1(x))
#         # print('x shape: ', x.shape) # x shape:  torch.Size([8, 32])
#         x = self.lin2(x)
#         # print('x shape: ', x.shape) # x shape:  torch.Size([8, 2])
#         return F.log_softmax(x, dim=-1), mc1 + mc2, o1 + o2

In [12]:
# The proposed code utlilizing Graph UNet

from torch_geometric.nn import GraphUNet
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=32):
        super(Net, self).__init__()
        self.conv1 = GraphUNet(in_channels, hidden_channels, hidden_channels, depth=3)
        num_nodes = ceil(0.5 * average_nodes)
        self.pool1 = Linear(hidden_channels, num_nodes)
        self.conv2 = DenseGraphConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, out_channels)
    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x, mask = to_dense_batch(x, batch)
        adj = to_dense_adj(edge_index, batch) 
        s = self.pool1(x)
        x, adj, mc1, o1 = dense_diff_pool(x, adj, s, mask)
        x = self.conv2(x, adj)
        x = x.mean(dim=1)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), mc1, o1 #, mc1 + mc2, o1 + o2

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, mc_loss, o_loss = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(out, data.y.view(-1)) + mc_loss + o_loss
        loss.backward()
        loss_all += data.y.size(0) * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)


@torch.no_grad()
def valid(loader):
    model.eval()
    correct = 0
    loss_all = 0

    for data in loader:
        data = data.to(device)
        pred, mc_loss, o_loss = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(pred, data.y.view(-1)) + mc_loss + o_loss
        loss_all += data.y.size(0) * loss.item()
        correct += pred.max(dim=1)[1].eq(data.y.view(-1)).sum().item()

    return loss_all / len(loader.dataset), correct / len(loader.dataset)

@torch.no_grad()
def test_eval(loader, model):
    model.eval()
    correct = 0
    loss_all = 0

    for data in loader:
        data = data.to(device)
        pred, mc_loss, o_loss = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(pred, data.y.view(-1)) + mc_loss + o_loss
        loss_all += data.y.size(0) * loss.item()
        correct += pred.max(dim=1)[1].eq(data.y.view(-1)).sum().item()

    return loss_all / len(loader.dataset), correct / len(loader.dataset)

best_val_acc = test_acc = 0
best_val_loss = float('inf')
start_patience = 50
patience = 50
for epoch in range(1, 200):
    train_loss = train(epoch)
    _, train_acc = valid(train_loader)
    val_loss, val_acc = valid(val_loader)
    if val_loss < best_val_loss and val_acc > best_val_acc:
        test_loss, test_acc = valid(test_loader)
        best_val_loss = val_loss
        best_val_acc = val_acc
        patience = start_patience
        eval_model = model
    else:
        patience -= 1
        if patience == 0:
            break
    print('Epoch: {:03d}, '
          'Train Loss: {:.3f}, Train Acc: {:.3f}, '
          'Val Loss: {:.3f}, Val Acc: {:.3f}, '
          'Test Loss: {:.3f}, Test Acc: {:.3f}'.format(epoch, train_loss,
                                                       train_acc, val_loss,
                                                       val_acc, test_loss,
                                                       test_acc))

fin_test_loss, fin_test_acc = test_eval(test_loader, eval_model)
print('Final Test Loss: {:.3f}, Test Acc: {:.3f}'.format(test_loss, test_acc))

Epoch: 001, Train Loss: 1.846, Train Acc: 0.714, Val Loss: 1.944, Val Acc: 0.739, Test Loss: 1.867, Test Acc: 0.739
Epoch: 002, Train Loss: 1.831, Train Acc: 0.724, Val Loss: 1.941, Val Acc: 0.748, Test Loss: 1.854, Test Acc: 0.748
Epoch: 003, Train Loss: 1.820, Train Acc: 0.734, Val Loss: 1.936, Val Acc: 0.730, Test Loss: 1.854, Test Acc: 0.748
Epoch: 004, Train Loss: 1.810, Train Acc: 0.730, Val Loss: 1.928, Val Acc: 0.721, Test Loss: 1.854, Test Acc: 0.748
Epoch: 005, Train Loss: 1.799, Train Acc: 0.733, Val Loss: 1.919, Val Acc: 0.721, Test Loss: 1.854, Test Acc: 0.748
Epoch: 006, Train Loss: 1.787, Train Acc: 0.735, Val Loss: 1.906, Val Acc: 0.721, Test Loss: 1.854, Test Acc: 0.748
Epoch: 007, Train Loss: 1.773, Train Acc: 0.736, Val Loss: 1.889, Val Acc: 0.730, Test Loss: 1.854, Test Acc: 0.748
Epoch: 008, Train Loss: 1.754, Train Acc: 0.735, Val Loss: 1.867, Val Acc: 0.730, Test Loss: 1.854, Test Acc: 0.748
Epoch: 009, Train Loss: 1.732, Train Acc: 0.734, Val Loss: 1.835, Val Ac